In [29]:

import os
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image


In [11]:
# Load the dataset
image_label_df = pd.read_csv('E:/task1/image_label.csv')
image_label_df['Image_name'] = image_label_df['Image_name']+ '.png'

In [12]:
# Data preprocessing
image_label_df['Image_name'] = 'E:/task1/train_test/' + image_label_df['Image_name']

In [13]:
# Splitting the dataset into training and testing sets
train_df, test_df = train_test_split(image_label_df, test_size=0.2, random_state=42)

In [14]:
# Image data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
# Define the image dimensions and batch size
img_width, img_height = 256, 256
batch_size = 32

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='Image_name',
    y_col='Plane',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

Found 1316 validated image filenames belonging to 4 classes.


In [16]:
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='Image_name',
    y_col='Plane',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)


Found 330 validated image filenames belonging to 4 classes.


In [17]:
# Define the neural network model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

In [18]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
epochs = 20  # You can increase this for better performance
model.fit(
    train_generator,
    steps_per_epoch=len(train_df) // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=len(test_df) // batch_size
)

loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy*100:.2f}%")

model.save('fetal_ultrasound_model.h5')

# Save the weights of the model's layers
model.save_weights('fetal_ultrasound_model_weights.h5')


Epoch 1/20
41/41 [==============================] - 473s 11s/step - loss: 1.5083 - accuracy: 0.3575 - val_loss: 1.0776 - val_accuracy: 0.5125
Epoch 2/20
41/41 [==============================] - 450s 11s/step - loss: 1.1146 - accuracy: 0.5179 - val_loss: 0.8387 - val_accuracy: 0.6906
Epoch 3/20
41/41 [==============================] - 460s 11s/step - loss: 0.9746 - accuracy: 0.6098 - val_loss: 0.7041 - val_accuracy: 0.7875
Epoch 4/20
41/41 [==============================] - 442s 11s/step - loss: 0.8589 - accuracy: 0.6760 - val_loss: 0.5750 - val_accuracy: 0.7969
Epoch 5/20
41/41 [==============================] - 414s 10s/step - loss: 0.8495 - accuracy: 0.6706 - val_loss: 0.5875 - val_accuracy: 0.8031
Epoch 6/20
41/41 [==============================] - 444s 11s/step - loss: 0.8028 - accuracy: 0.7002 - val_loss: 0.6413 - val_accuracy: 0.7875
Epoch 7/20
41/41 [==============================] - 423s 10s/step - loss: 0.7806 - accuracy: 0.7188 - val_loss: 0.4932 - val_accuracy: 0.8281
Epoch 

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model


In [ ]:
# Save the model for future use


In [20]:
# Load the saved model
model = tf.keras.models.load_model('fetal_ultrasound_model.h5')

In [22]:
# Load the test dataset and labels
test_df = pd.read_csv(r'E:/task1/image_label.csv')
test_df['Image_name'] = test_df['Image_name']+ '.png'
test_df['Image_name'] = r'E:/task1/train_test/' + test_df['Image_name']


In [23]:
# Data preprocessing for the test set
test_datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='Image_name',
    y_col='Plane',
    target_size=(256, 256),  # Assuming the same image dimensions as used during training
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1646 validated image filenames belonging to 4 classes.


In [24]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy*100:.2f}%")

52/52 [==============================] - 161s 3s/step - loss: 0.4526 - accuracy: 0.8396
Test Loss: 0.4526, Test Accuracy: 83.96%


In [22]:
# Create a directory to save classified images
output_directory = 'E:/task1/classified_images'
os.makedirs(output_directory, exist_ok=True)

In [23]:
# Get class labels and indices
class_labels = {v: k for k, v in test_generator.class_indices.items()}

hypotheses = {
    'abdomen': 'This image is likely of the abdomen.',
    'thorax': 'This image is likely of the thorax.',
    'brain': 'This image is likely of the brain.',
    'femur': 'This image is likely of the femur.'
}

In [ ]:
# Iterate through test data and save classified images to respective folders
for i in range(len(test_generator.filenames)):
    image_path = test_generator.filepaths[i]
    true_label = test_generator.labels[i]
    
    # Load and preprocess the image for prediction
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize
    
    # Make a prediction
    predicted_label = np.argmax(model.predict(img_array), axis=-1)[0]

    # Create subdirectories for each class label
    label_directory = os.path.join(output_directory, class_labels[true_label])
    os.makedirs(label_directory, exist_ok=True)

    # Copy the image to the corresponding folder
    image_name = os.path.basename(image_path)
    shutil.copy(image_path, os.path.join(label_directory, image_name))

print("Classified images saved to respective folders.")

To test whether the model performs well or not, you can provide another image as input

In [30]:
# Load the saved model
model = tf.keras.models.load_model('fetal_ultrasound_model.h5')

# Define a function for classification
def classify_image(image_path):
    # Load and preprocess the input image
    img = image.load_img(image_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize
    
    # Make a prediction
    prediction = model.predict(img_array)
    
    # Map predicted class index to class label
    class_labels = {0: 'abdomen', 1: 'thorax', 2: 'brain', 3: 'femur'}
    predicted_class = class_labels[np.argmax(prediction)]
    
    return predicted_class

# Example usage:
image_path = r'E:\task1\External Test images\Patient01600_Plane6_1_of_8.png'
predicted_label = classify_image(image_path)
print(f"The input image is classified as: {predicted_label}")

1/1 [==============================] - 1s 666ms/step
The input image is classified as: femur
